In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import catboost as cat

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 777

In [3]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df['mol'] = train_df['mol'].apply(lambda x: Chem.AddHs(x))
  train_df['num_of_atoms'] = train_df['mol'].apply(lambda x: x.GetNumAtoms())
  train_df['num_of_heavy_atoms'] = train_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  test_df['mol'] = test_df['mol'].apply(lambda x: Chem.AddHs(x))
  test_df['num_of_atoms'] = test_df['mol'].apply(lambda x: x.GetNumAtoms())
  test_df['num_of_heavy_atoms'] = test_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  # 중복제거
  def canonize(mol):
    return Chem.MolToSmiles(Chem.MolFromSmiles(mol), isomericSmiles=True, canonical=True)

  canon_smile = []
  for molecule in train_df['SMILES']:
    canon_smile.append(canonize(molecule))
  
  train_df['canon_smiles'] = canon_smile
  
  ind = train_df.index[train_df['canon_smiles'].duplicated()]
  train_df = train_df.drop(ind)
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["HLM"].values
  
  return train_features, target, test_features
  

In [4]:
train_x, train_y, test_x = feature_engineer()

In [5]:
def create_model(param):
  return cat.CatBoostRegressor(
    random_state=RANDOM_SEED,
    verbose=False,
    **param
  )

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50, verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 2000, 20000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
    }

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='cat_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_cat, n_trials=70)

[I 2023-09-07 01:48:32,022] A new study created in memory with name: cat_tuning


0:	learn: 35.5279355	test: 35.5279355	test1: 36.0666937	best: 36.0666937 (0)	total: 179ms	remaining: 14m 9s
500:	learn: 30.0182366	test: 30.0182366	test1: 32.2402494	best: 32.2402494 (500)	total: 972ms	remaining: 8.24s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.23388499
bestIteration = 501

Shrink model to first 502 iterations.
Fold: 1/5 score = 1039.02335
0:	learn: 35.5305554	test: 35.5305554	test1: 36.0379248	best: 36.0379248 (0)	total: 1.52ms	remaining: 7.22s
500:	learn: 29.9145148	test: 29.9145148	test1: 32.6462500	best: 32.6462500 (500)	total: 794ms	remaining: 6.73s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.54894899
bestIteration = 674

Shrink model to first 675 iterations.
Fold: 2/5 score = 1059.43409
0:	learn: 35.6769549	test: 35.6769549	test1: 35.3895981	best: 35.3895981 (0)	total: 1.49ms	remaining: 7.07s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.80077973
bestIteration = 198

Shrink model to first 1

[I 2023-09-07 01:48:51,895] Trial 0 finished with value: 1032.923542228917 and parameters: {'iterations': 4748, 'learning_rate': 0.09140462630254884, 'depth': 1, 'subsample': 0.5138743074480719, 'min_data_in_leaf': 86, 'colsample_bylevel': 0.9306471953623999}. Best is trial 0 with value: 1032.923542228917.


0:	learn: 34.9444791	test: 34.9444791	test1: 35.5033658	best: 35.5033658 (0)	total: 5.16ms	remaining: 1m 17s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.02231115
bestIteration = 137

Shrink model to first 138 iterations.
Fold: 1/5 score = 1025.42842
0:	learn: 35.7162147	test: 35.7162147	test1: 36.2842622	best: 36.2842622 (0)	total: 4.51ms	remaining: 1m 8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.73289551
bestIteration = 40

Shrink model to first 41 iterations.


[I 2023-09-07 01:48:59,430] Trial 1 pruned. 


...

0:	learn: 35.7966121	test: 35.7966121	test1: 36.3527180	best: 36.3527180 (0)	total: 41.5ms	remaining: 7m 35s
500:	learn: 24.4717227	test: 24.4717227	test1: 31.9864127	best: 31.9739076 (453)	total: 25s	remaining: 8m 43s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.97390761
bestIteration = 453

Shrink model to first 454 iterations.
Fold: 1/5 score = 1022.33078
0:	learn: 35.8093158	test: 35.8093158	test1: 36.3366294	best: 36.3366294 (0)	total: 45ms	remaining: 8m 13s
500:	learn: 24.6423688	test: 24.6423688	test1: 32.1351375	best: 32.1259679 (494)	total: 27.6s	remaining: 9m 36s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.99758087
bestIteration = 720

Shrink model to first 721 iterations.
Fold: 2/5 score = 1023.84519
0:	learn: 35.9730684	test: 35.9730684	test1: 35.6423149	best: 35.6423149 (0)	total: 44.9ms	remaining: 8m 13s
500:	learn: 24.4130694	test: 24.4130694	test1: 32.3238067	best: 32.3159524 (498)	total: 26.8s	remaining: 9m 20s
Stopped b

[I 2023-09-07 05:23:08,868] Trial 68 finished with value: 996.5418428697424 and parameters: {'iterations': 10979, 'learning_rate': 0.02290568048368033, 'depth': 9, 'subsample': 0.621047614338184, 'min_data_in_leaf': 96, 'colsample_bylevel': 0.5735297772378283}. Best is trial 32 with value: 989.6882444196902.


Fold: 5/5 score = 946.05690
0:	learn: 35.8907998	test: 35.8907998	test1: 36.4398325	best: 36.4398325 (0)	total: 87.8ms	remaining: 17m 21s
500:	learn: 29.5546890	test: 29.5546890	test1: 32.4405240	best: 32.4400437 (498)	total: 43.7s	remaining: 16m 31s
1000:	learn: 27.1144883	test: 27.1144883	test1: 32.0501454	best: 32.0501454 (1000)	total: 1m 27s	remaining: 15m 48s
1500:	learn: 23.9631685	test: 23.9631685	test1: 31.8356385	best: 31.8349698 (1499)	total: 2m 11s	remaining: 15m 8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.7965539
bestIteration = 1585

Shrink model to first 1586 iterations.
Fold: 1/5 score = 1011.02085
0:	learn: 35.8962983	test: 35.8962983	test1: 36.4042964	best: 36.4042964 (0)	total: 95.1ms	remaining: 18m 49s
500:	learn: 29.5693813	test: 29.5693813	test1: 32.6941317	best: 32.6938875 (499)	total: 50.2s	remaining: 18m 58s
1000:	learn: 27.1457652	test: 27.1457652	test1: 32.3282248	best: 32.3282248 (1000)	total: 1m 40s	remaining: 18m 14s
1500:	learn:

[I 2023-09-07 05:37:26,735] Trial 69 finished with value: 995.3393608070016 and parameters: {'iterations': 11870, 'learning_rate': 0.006940569752777088, 'depth': 10, 'subsample': 0.7364016206318245, 'min_data_in_leaf': 96, 'colsample_bylevel': 0.6061333646326709}. Best is trial 32 with value: 989.6882444196902.


Fold: 5/5 score = 937.74869


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'iterations': 19992, 'learning_rate': 0.013848719872751707, 'depth': 10, 'subsample': 0.9236649543282481, 'min_data_in_leaf': 92, 'colsample_bylevel': 0.28087799719234086}


In [11]:
best_param = {
  'iterations': 19992,
  'learning_rate': 0.013848719872751707,
  'depth': 10,
  'subsample': 0.9236649543282481,
  'min_data_in_leaf': 92,
  'colsample_bylevel': 0.28087799719234086
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

0:	learn: 35.8491442	test: 35.8491442	test1: 36.3988443	best: 36.3988443 (0)	total: 89.7ms	remaining: 29m 53s
500:	learn: 27.4702340	test: 27.4702340	test1: 32.1295803	best: 32.1295803 (500)	total: 41.3s	remaining: 26m 45s
1000:	learn: 21.0213634	test: 21.0213634	test1: 31.7098373	best: 31.7072120 (998)	total: 1m 21s	remaining: 25m 51s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.70440803
bestIteration = 1010

Shrink model to first 1011 iterations.
Fold: 1/5 score = 1005.16950
0:	learn: 35.8581400	test: 35.8581400	test1: 36.3722080	best: 36.3722080 (0)	total: 93.1ms	remaining: 31m 1s
500:	learn: 27.4898796	test: 27.4898796	test1: 32.3985563	best: 32.3974794 (499)	total: 45.3s	remaining: 29m 23s
1000:	learn: 21.3660238	test: 21.3660238	test1: 32.0570496	best: 32.0570496 (1000)	total: 1m 30s	remaining: 28m 44s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.95595367
bestIteration = 1270

Shrink model to first 1271 iterations.
Fold: 2/5 score =

In [12]:
for idx, model in enumerate(models):
  model.save_model(f"model/catboost_HLM_v1_fold_{idx}.model")

In [13]:
submit = pd.read_csv("../data/sample_submission.csv")
submit["HLM"] = np.mean(y_tests, axis=0)
submit.to_csv("catboost_optuna.csv", index=False)